In [ ]:
import os
import s3fs
import zarr

from typing import Union

import numpy as np
import dask.array as da
import astropy.units as u
from sunpy.net import Fido, attrs as a
from sunpy.map import Map


def s3_connection(path_to_zarr: os.path) -> s3fs.S3Map:
    """
    Instantiate connection to aws for a given path `path_to_zarr`
    """
    return s3fs.S3Map(
        root=path_to_zarr,
        s3=s3fs.S3FileSystem(anon=True),
        # anonymous access requires no credentials
        check=False,
    )


def load_single_aws_zarr(
    path_to_zarr: os.path,
    cache_max_single_size: int = None,
) -> Union[zarr.Array, zarr.Group]:
    """
    load zarr from s3 using LRU cache
    """
    return zarr.open(
        zarr.LRUStoreCache(
            store=s3_connection(path_to_zarr),
            max_size=cache_max_single_size,
        ),
        mode="r",
    )

In [ ]:
root = load_single_aws_zarr(
    path_to_zarr="s3://gov-nasa-hdrl-data1/contrib/fdl-sdoml/fdl-sdoml-v2/sdomlv2_hmi.zarr",
)
data_bx = root["2010"]["Bx"]
sorted_indices = np.argsort(data_bx.attrs["T_OBS"])
img_index = sorted_indices[0]
selected_headr = {keys: values[img_index] for keys, values in data_bx.attrs.items()}
selected_image = da.from_array(data_bx)[img_index, :, :]
smap_ml_bx = Map((np.array(selected_image), selected_headr))

data_by = root["2010"]["By"]
sorted_indices = np.argsort(data_by.attrs["T_OBS"])
img_index = sorted_indices[0]
selected_headr = {keys: values[img_index] for keys, values in data_by.attrs.items()}
selected_image = da.from_array(data_by)[img_index, :, :]
smap_ml_by = Map((np.array(selected_image), selected_headr))

data_bz = root["2010"]["Bz"]
sorted_indices = np.argsort(data_by.attrs["T_OBS"])
img_index = sorted_indices[0]
selected_headr = {keys: values[img_index] for keys, values in data_bz.attrs.items()}
selected_image = da.from_array(data_bz)[img_index, :, :]
smap_ml_bz = Map((np.array(selected_image), selected_headr))

NameError: name 'data' is not defined

In [ ]:
tr = a.Time("2010-05-13T00:00:00", "2010-05-13T00:00:12")
aia_query = Fido.search(tr & a.Instrument.aia & a.Wavelength(94*u.angstrom))
filelist_fd = Fido.fetch(aia_query, path="data/{instrument}")
smap_fd = Map(filelist_fd[0])